1\. Write a function that converts number representation (bin<->dec<->hex)

In [37]:
import math
hex_dict = {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6: '6', 7:'7', 8:'8', 9:'9', 10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f', 16: 'g'}

def conv(num, da, a):
    num_str = str(num)
    dec = 0
    power = len(num_str)
    for digit in num_str:
        power-=1
        dec += int(digit, da)*(da**(power))
    result = ''
    while(dec):
        result = hex_dict[dec % a] + result
        dec = math.floor(dec/a)
    return result

x = 11111
print(conv(x, 2, 16))
print(conv(x, 2, 10))

1f
31


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [56]:
def to_float(word):
    sign = 1 if word[0]=='0' else -1
    bias = 127
    power = int(conv(word[1:9], 2, 10)) - bias
    mantissa = '1' + word[9:32]
    result = 0
    for digit in mantissa:
        if digit == '1':
            result += 2**power
        power -= 1
    return sign*result

prova = '1100000010110000000000000000000000'
print(to_float(prova))

-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [67]:
numo = 1
numu = 1
of = 0
uf = 0
while( numo*2 > numo ):
    of = numo
    numo *= 2
while( numu/2 < numu):
    uf = numu
    numu /= 2
print( "Overflow:", of, "Underflow:", uf, sep = ' ' )

KeyboardInterrupt: 

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
